Importing libraries

In [14]:
import pandas as pd

import psycopg2

Now we procede to read and import all tables.

In [15]:
data_friends = pd.read_csv('./tables/friends_table.csv')
print(data_friends.shape)
data_friends.sample(8)

(9402, 2)


,Friend 1,Friend 2
1397,149,776
1610,171,53
8888,948,13
1185,125,722
9158,978,539
4849,524,872
6811,724,230
5649,609,959


In [16]:
data_post = pd.read_csv('./tables/posts_table.csv')
print(data_post.shape)
data_post.sample(8)

(13158, 3)


,User,Post Type,Post Date
8270,633,Text,1588165715
6199,472,Status_Change,1588165818
2024,158,Gif,1588165700
1614,128,Video,1588165450
4631,354,Text,1588163340
11194,856,Status_Change,1588167598
11360,868,Image,1588166172
344,26,Text,1588164967


In [17]:
data_reactions = pd.read_csv('./tables/reactions_table.csv')
print(data_reactions.shape)
data_reactions.sample(8)

(26365, 3)


,User,Reaction Type,Reaction Date
6242,8729,Comment,1588166524
16334,3592,Emoticon,1588161989
13033,6988,Emoticon,1588161128
11484,734,Emoticon,1588166429
2336,679,Comment,1588167753
11996,1167,Comment,1588166520
22689,714,Comment,1588162735
3437,667,Like,1588165650


In [18]:
data_user = pd.read_csv('./tables/user_table.csv')
print(data_user.shape)
data_user.sample(8)

(1000, 4)


,Surname,Name,Age,Subscription Date
55,Wellington,Zoe,34,1588157419
936,Bond,Franz,30,1588159326
497,Kirk,Zoe,21,1588150514
925,Roth,Simon,40,1588162711
287,Mueller,Francine,20,1588157962
902,Beierlorzer,Simon,37,1588157275
316,Kirk,Zoe,24,1588145636
67,Pomme,Andreas,35,1588168007


the relation with tables is given by user id, from table 'data_user'. Going to check if user has the same range of values in each table, so we can ensure that the index from 'data_user' table is the relational key between tables.

In [19]:
# Grouping with 'Friend 1' values, to check if length is equal to 'data_user' index.
print('Lenght Friend 1:',len(data_friends['Friend 1'].value_counts()))

# Same with 'Friend 2'.
print('Lenght Friend 2:',len(data_friends['Friend 2'].value_counts()))

Lenght Friend 1: 1000
Lenght Friend 2: 1000


Here i checked that the values on friends tables matches with the index of users table. 

In 'data_user' table the index is my relation with all tables.

In [20]:
# Creating new column, so it can be easyer to perform unions.
data_user['User'] = data_user.index
data_user.sample(3)

,Surname,Name,Age,Subscription Date,User
197,Stephenson,Ali,24,1588159417,197
486,Kirk,Francine,37,1588161267,486
935,Pomme,Sarah,32,1588155083,935
